### 1 - Import test data

In [7]:
from data import data_connector, process_data
from trader import auth, buy_sell
import pandas as pd
import streamlit as st

st.title("Hello World")

my_trader = buy_sell.trade()
my_auth = auth.authoriser()
my_connector = data_connector.api_connector()
my_processor = process_data.processor()

fifteen_min_dataF = my_connector.yf_get(60)
#this is a rest API - it only gets data every 15mins and acts off what is finds


#fifteen_min_dataF.iloc[:,:]
#fifteen_min_dataF.Open.iloc
#one_day_dataF = yf.download('EURUSD=X', period='1y', interval='1d')

fifteen_min_dataF = my_processor.add_sma(fifteen_min_dataF, [50,200])

#fifteen_min_dataF = my_processor.add_sma(fifteen_min_dataF, 50)
#fifteen_min_dataF = my_processor.add_sma(fifteen_min_dataF, 200)

#fifteen_min_dataF["sma_50"] = ta.trend.SMAIndicator(fifteen_min_dataF["Close"], 50).sma_indicator()
#fifteen_min_dataF["sma_200"] = ta.trend.SMAIndicator(close=fifteen_min_dataF["Close"], window=200).sma_indicator()


2024-09-21 10:03:27.964 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-21 10:03:29.230 
  command:

    streamlit run c:\Users\Angus\anaconda3\envs\trading-env\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-21 10:03:29.231 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
[*********************100%%**********************]  1 of 1 completed


### 2 - Define your signal functions

In [8]:
def og_signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

In [9]:
def sma_signal_generator(df):
    #very simple process
    
    sma_50 = df.sma_50
    sma_200 = df.sma_200
    #print(f"200 + {sma_200}")
    #print(sma_50)
    #bearish
    if sma_50 > sma_200:
        return 2
    #bullish
    elif sma_50 < sma_200:
        return 1
    #none
    else:
        return 0

#sma_signal_generator(fifteen_min_dataF)

#  need to remember the SMA is still a changing value for each new candle - needs a column


#### add signal column

In [26]:
#signal = []
#signal.append(0)
#for i in range(1,len(fifteen_min_dataF)):
#    df = fifteen_min_dataF[i-1:i+1]
#    signal.append(og_signal_generator(df))
##og_signal_generator(data)
#fifteen_min_dataF["signal"] = signal
#    

In [10]:
sma_signal = []
sma_signal.append(0)
for i in range(1,len(fifteen_min_dataF)):
    df = fifteen_min_dataF.iloc[i]
    #print(df)
    sma_signal.append(sma_signal_generator(df))
#og_signal_generator(data)
fifteen_min_dataF["sma_signal"] = sma_signal


In [11]:
fifteen_min_dataF.sma_signal.value_counts()
fifteen_min_dataF.iloc[:, :]
fifteen_min_dataF.to_csv("dataf.csv", index = False)


In [29]:
#checking the return on the 1 day data download

#signal = []
#signal.append(0)
#for i in range(1,len(one_day_dataF)):
#    df = one_day_dataF[i-1:i+1]
#    signal.append(og_signal_generator(df))
##og_signal_generator(data)
#
#one_day_dataF["signal"] = signal
#one_day_dataF.signal.value_counts()
#one_day_dataF.iloc[:, :]

In [30]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails


In [31]:
def str_to_bool(s):
    if s.lower() in ['true', '1', 't', 'y', 'yes']:
        return True
    elif s.lower() in ['false', '0', 'f', 'n', 'no']:
        return False
    else:
        raise ValueError(f"Cannot convert {s} to boolean, please input a formattable string")
    

In [35]:
is_live = False
has_prompted = False


def auth_deets(is_live, type_:str,):
    global has_prompted
    if not has_prompted:
        is_live = str_to_bool(s=input("Would you like to run on your LIVE account? if n then will be DEMO (y/n)"))
        has_prompted = True
    if is_live:
        if type_ == "token":
            access_token='84aa14ef906b9b989c64bff10d032b30-7fefd28cd58969179eaf00aad7c2baa5'# you need token here generated from OANDA account for LIVE
            return access_token
        elif type_ == "id":
            accID = '001-004-12283531-001' #my account ID here from oanda for LIVE
            return accID
    else:
        if type_ == "token":
            access_token='eac0b37f2067f37b1bb9884dfb473e6b-e204948541e0c8a2ba8a967492225ca7' # DEMO TOKEN
            return access_token
        elif type_ == "id":
            accID = '101-004-29576199-001' #my acc ID for the DEMO
            return accID

In [39]:
my_candles = my_connector.get_candles(False, 3, token=my_auth.auth_deets(is_live, "token", has_prompted=has_prompted), pair="EUR_USD", interval=1, )

#from config import access_token, accountID
x=0
def M15_get_candles(n, access_token=my_auth.auth_deets(is_live, "token"), ):
    client = CandleClient(access_token, real=is_live)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15) #15 is the time frame (15minutes)
    candles = collector.grab(n)
    return candles#<- contain info like open and close price

#in the "oanda_candles" in the env packages it shows the bot will either use the api url for real or practice trading depending on the real statement in the client line

#candles = M15_get_candles(3)
#for candle in my_candles:
#    print(float(str(candle.bid.o))>1)
# TRUE outputs tell us the last candles have an open price above 1


TypeError: authoriser.auth_deets() missing 1 required positional argument: 'has_prompted'

#### 3 - Connect to the market and execute trades

In [ ]:
def og_M15_trading_job():
    candles = M15_get_candles(3) #last candle is still open (its a current price so the close price is fluctuating)
    
    format_tuple = my_processor.format_columns(candles)

    dfstream = format_tuple[0]
    
    candle = format_tuple[1]
    
    signal = og_signal_generator(dfstream.iloc[:-1,:])
    
    # EXECUTING ORDERS

    accID = my_auth.auth_deets(is_live, "id")
    access_token = my_auth.auth_deets(is_live, "token")
    client = API(access_token)
    
    #-----------------------------------------------------------------
    # all this defines stop loss and stop profit     
    p_l_values = my_trader.p_l_stops(dfstream, 2., candle=candle)
    print(type(candle))
    print(dfstream.iloc[:-1,:])
    print(p_l_values)
    
    my_trader.buy_sell(signal, client, accID, p_l_values)
    
    #Sell
    #if signal == 1:
    #    mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
    #    r = orders.OrderCreate(accountID = accID, data=mo.data)
    #    rv = client.request(r)
    #    print(rv)
    #Buy
    #elif signal == 2:
    #    mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
    #    r = orders.OrderCreate(accountID = accID, data=mo.data)
    #    rv = client.request(r)
    #    print(rv)

In [ ]:
def sma_M15_trading_job():
    candles = M15_get_candles(200) #last candle is still open (its a current price so the close price is fluctuating)
    
    format_tuple = my_processor.format_columns(candles)

    dfstream = format_tuple[0]
    
    candle = format_tuple[1]

    windows = [50,200]

    dfstream = my_processor.add_sma(dfstream, sma_windows=windows)



    #return dfstream.iloc[-1,]

    signal = sma_signal_generator(dfstream.iloc[-1,])
    
    # EXECUTING ORDERS

    accID = my_auth.auth_deets(is_live, "id")
    access_token = my_auth.auth_deets(is_live, "token")

    client = API(access_token)

    #-----------------------------------------------------------------
    # all this defines stop loss and stop profit     
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    print(type(candle))
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)

    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID = accID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID = accID, data=mo.data)
        rv = client.request(r)
        print(rv)

: 

### 4 - Executing orders automatically with a scheduler

In [ ]:
#sma_M15_trading_job()

scheduler = BlockingScheduler()
scheduler.add_job(sma_M15_trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
scheduler.start()